## POS

In [ ]:
def POS_pre(sources,table_name,columnname, base_table_member,threshold = 0.1,previous_results = {}):
    # try:
        engine = create_engine(db_connection_string)
        today = date.today()
            # print(current_counts)
        # today = datetime.now().strftime('%m%d%Y')
        query = f"""
                Select {columnname} , count(distinct [{base_table_member}]) as counts from
                {sources}.{table_name} group by [{columnname}]
                """
        with engine.connect() as conn:
            current_year_results = conn.execute(text(query)).fetchall()
        
        current_year_total = sum(row[1] for row in current_year_results)
        current_year_counts = {row[0]: row[1] for row in current_year_results}
        today = date.today().strftime('%m%d%Y')
        
        if today in previous_results:
            previous_counts = previous_results[today]
        else:
            if previous_results:
                latest_date = max(previous_results.keys())
                       
                previous_counts = previous_results[latest_date]
            else:
                previous_counts = current_year_counts.copy()
                
       
                
        error_flag = 0
        error_flag_list = []
        for category,current_count in current_year_counts.items():
            previous_count = previous_counts.get(category,0)
            if previous_count>0:
                percentage_change = abs((current_count - previous_count)/previous_count)
            else:
                percentage_change = current_count
            if percentage_change>threshold:
                error_flag = 1
                error_flag_list.append(1)
                
            else:
                error_flag_list.append(0)
            
                      
        previous_results[today] = current_year_counts
        
                # print(current_year_counts)
                # print("--------")
                # print(previous_results)
        return current_year_counts,previous_results,error_flag ,error_flag_list
    # except FileNotFoundError:
    #     return json.dumps({"error": "Excel file not found."})
    # except SQLAlchemyError as e:
    #     return json.dumps({"error": f"Database error: {str(e)}"})
    # except Exception as e:
    #     return json.dumps({"error": f"An error occurred: {e}"})
        

In [ ]:
def POS_CHECK_main(sources,final_list,df,measure):
    # measure_to_value_filtered = measure_to_value[measure_to_value['Measure ID']==measure]
    # df = pd.read_csv('config3.csv')
    age_check_rows = df[(df['Function_Type'] == 'POS') & (df['DataSource'] == sources)]
    engine = create_engine(db_connection_string)
    value_set_final_list = []
    source_rows = age_check_rows[age_check_rows['DataSource'] == sources]
    for index, row in source_rows.iterrows():
        value_set_final_dict = {}
        test_case_id = row['TC_ID']
        table_name = row['TableName']
        columnname = row['Column']  
        base_table_member = row['BaseTableMember']
        
        # filtered_data = [d for d in final_list]
        result = None
        # for d in filtered_data:
        #     if code_system in d['Previous Count']:
        if len(final_list) !=0:
            result = final_list[0]
                
        if result is not None:  
            previous_results = {}
            date_prev = result['DVE Run Date']
            previous_results[date_prev] = result['Previous Count']
        else:
            previous_results = {}
        current_counts,previous_results,error_flag,error_flag_list = POS_pre(sources,table_name,columnname,base_table_member, threshold = 0.1,previous_results = {})
        
        value_set_final_dict['DVE Run Date'] = date.today().strftime('%m%d%Y')
        value_set_final_dict['Test Case ID'] = test_case_id
        value_set_final_dict['Measure'] = measure
        value_set_final_dict['Function'] = "POS"
        value_set_final_dict['Comparision'] = "% of values with inconsistent POS"
        value_set_final_dict['Datasource'] = sources
        value_set_final_dict['TableName'] = table_name
        value_set_final_dict['ColumnName'] = columnname
        value_set_final_dict['Current Count'] = current_counts
        value_set_final_dict['Previous Count'] = previous_results
        value_set_final_dict['Error Flag Individual'] = error_flag_list
        value_set_final_dict['Error Flag'] = error_flag
        value_set_final_list.append(value_set_final_dict)
    return value_set_final_list
        


## LAB Test

In [ ]:
def LAB_RESULTS_CHECK(source,df,measure):
    # try:
        # df = pd.read_csv('config3.csv')
        lab_df = pd.read_csv('lionic.csv')
        for col in ['Acceptable Value (MIN)','Acceptable Value (MAX)']:
            lab_df[col] = pd.to_numeric(lab_df[col],errors = 'coerce')
    
        # print(lab_files)
       # universe_check_rows = df[(df['Function_Type'] == 'Lab_Check') & (df['DataSource'] == source)]
        engine = create_engine(db_connection_string)
        value_list = []
        # print(df)
        source_rows = df[df['DataSource'] == source]
        for index, row in source_rows.iterrows():
            value_set_final_dict = {}
            test_case_id = row['TC_ID']
            table_name = row['TableName']
            columnname = row['Column']  
            base_table_member = row['BaseTableMember']
            check_column = row['check_column']
                
            with engine.connect() as conn:
                members_ids = pd.read_sql_query(f"Select  {columnname},{check_column} from dbo.LAB_IN",conn)
                # print(members_ids)
            # print(members_ids.head())
            merged_df = pd.merge(members_ids,lab_df,on = 'LOINC',how = 'left')
            error_flag = []
            for i,r in merged_df.iterrows():
                if (r['LAB_VALUE'] < r['Acceptable Value (MIN)']) and (r['LAB_VALUE'] > r['Acceptable Value (MAX)']):
                    error_flag.append(1)
                else:
                    error_flag.append(0)
            merged_df['Error'] = error_flag
            result_dict = {}
            for loinc in merged_df['LOINC'].unique():
                loinc_data = merged_df[merged_df['LOINC']==loinc]
                outside_count = ((loinc_data['LAB_VALUE']<loinc_data['Acceptable Value (MIN)'])|
                                (loinc_data['LAB_VALUE']>loinc_data['Acceptable Value (MAX)'])).sum()
                result_dict[loinc] = {
                    'Acceptable Min Value' : loinc_data['Acceptable Value (MIN)'].iloc[0],
                    'Acceptable Max Value' : loinc_data['Acceptable Value (MAX)'].iloc[0],
                    'Number of Values Outside Min Max Value' : outside_count,
                    'Error flag' : 1 if outside_count > 0 else 0
                }
                value_list.append(result_dict)
            return value_list
    

    # except FileNotFoundError:
    #     return json.dumps({"error": "Excel file not found."})
    # except SQLAlchemyError as e:
    #     return json.dumps({"error": f"Database error: {str(e)}"})
    # except Exception as e:
    #     return json.dumps({"error": f"An error occurred: {str(e)}"})


db_connection_string = 'mssql+pyodbc://zsaudit:9v]3>SYv1C20@lhp-db-copy.database.windows.net:1433/HEDIS_2023_Copy?driver=ODBC+Driver+17+for+SQL+Server'
# print(UNIVERSE_CHECK('dbo'))
# df = pd.read_csv('config3.csv')
# df = df[]
# li = LAB_RESULTS_CHECK('dbo',df,'CBP')
# li